# Leveraging Agents with LangChain

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. You can also run on a local setup, as long as you have the right IAM credentials to invoke the Claude model via Bedrock*

### Building  - Key Elements

The first process in a building a contextual-aware chatbot is to identify the tools which can be called by the LLM's. 

Second process is the user request orchestration , interaction,  invoking and returing the results

### Architecture [Weather lookup]
We Search and look for the Latitude and Longitude and then invoke the weather app to get predictions

![Amazon Bedrock - Agents Interface](./images/weather.jpg)

#### Please un comment and install the libraries below if you do not have these 

In [3]:
!pip install langchain==0.1.17
!pip install faiss-cpu==1.8.0
!pip install pypdf
!pip install langchain_aws
!pip install boto3
!pip install langchain-anthropic

  Using cached langchain-0.1.17-py3-none-any.whl.metadata (13 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.60-py3-none-any.whl.metadata (13 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux

# Set up Bedrock client

In [4]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import load_tools, initialize_agent, Tool, AgentType
from langchain.llms.bedrock import Bedrock
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import tool, StructuredTool
from typing import Optional, List, Any
import requests
import json
import re
from PIL import Image
from io import BytesIO, StringIO
from IPython.display import display, Markdown, Pretty
from langchain.callbacks.manager import CallbackManagerForLLMRun
import sys
import textwrap
import boto3
import os
from pprint import pprint

In [5]:
region = os.environ.get("AWS_REGION")
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

# Get Bedrock LLM

In [6]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-v2"
llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"temperature": 0.1},
)

# Building an Agent

# 1. Tooling

## 1.1 Weather tool

In [7]:
@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params, headers=headers).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

##  Try out the OSM API to retrieve coordinates

In [8]:
# get the latitude and longitude for a certain location
coordinates = get_lat_long("Zurich")
if not coordinates:
    print("Something went wrong")
else:
    print(coordinates)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'latitude': '47.3744489', 'longitude': '8.5410422'}


# Task 1: The function below is incomplete. pls fix ;)

In [9]:
# Please fill in the missing pieces of the function definition to create a get_weather function that calls the open-meteo API for getting the current weather based on input coordinates (latitude and longitude)
# use the requests library to call the URI and
# return the response as dictionary

In [10]:
@tool ("get_weather")
def get_weather(latitude:str, longitude: str) -> dict:
  """Returns weather data for a given latitude and longitude."""
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  return response.json()

In [11]:
# get the weather for the retrived coordinates
response = get_weather(coordinates)
response

{'latitude': 47.38,
 'longitude': 8.539999,
 'generationtime_ms': 0.06389617919921875,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 416.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-05-21T19:45',
  'interval': 900,
  'temperature': 14.7,
  'windspeed': 5.1,
  'winddirection': 176,
  'is_day': 0,
  'weathercode': 80}}

## Ask the LLM how the weather in Zurich is

In [12]:
llm.invoke("Can you check the weather in Zurich for me?")

AIMessage(content='Unfortunately I don\'t have direct access to real-time weather data for specific locations. However, I can suggest searching the web or using a weather app to look up the current weather conditions in Zurich. You could search for something like "Zurich weather" to find weather reports and forecasts for that city. Let me know if you need any other help finding weather information!', additional_kwargs={'usage': {'prompt_tokens': 20, 'completion_tokens': 80, 'total_tokens': 100}}, response_metadata={'model_id': 'anthropic.claude-v2', 'usage': {'prompt_tokens': 20, 'completion_tokens': 80, 'total_tokens': 100}}, id='run-8e09e9a0-85c2-44cc-8c53-841f1cd29bb1-0')

## It is not able ot answer, so let's add a tool to make it capable to do so

## 1.2 Provide weather tool to LLM

In [13]:
tools_list = [get_lat_long, get_weather]
for tools_s in tools_list:
    display(Pretty((f"Tool:name={tools_s.name}::args={tools_s.args}:: discription={tools_s.description}::")))

Tool:name=get_lat_long::args={'place': {'title': 'Place', 'type': 'string'}}:: discription=get_lat_long(place: str) -> dict - Returns the latitude and longitude for a given place name as a dict object of python.::

Tool:name=get_weather::args={'latitude': {'title': 'Latitude', 'type': 'string'}, 'longitude': {'title': 'Longitude', 'type': 'string'}}:: discription=get_weather(latitude: str, longitude: str) -> dict - Returns weather data for a given latitude and longitude.::

In [14]:
llm_agent = initialize_agent(
    tools_list, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=3,
    return_intermediate_steps=True,
    max_execution_time=60
)
pprint(f"The prompt:template:used:was ---- > \n\n{llm_agent.agent.llm_chain.prompt}\n")

('The prompt:template:used:was ---- > \n'
 '\n'
 "input_variables=['agent_scratchpad', 'input'] "
 'messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], '
 "template='Respond to the human as helpfully and accurately as possible. You "
 'have access to the following tools:\\n\\nget_lat_long: get_lat_long(place: '
 'str) -> dict - Returns the latitude and longitude for a given place name as '
 "a dict object of python., args: {{\\'place\\': {{\\'title\\': \\'Place\\', "
 "\\'type\\': \\'string\\'}}}}\\nget_weather: get_weather(latitude: str, "
 'longitude: str) -> dict - Returns weather data for a given latitude and '
 "longitude., args: {{\\'latitude\\': {{\\'title\\': \\'Latitude\\', "
 "\\'type\\': \\'string\\'}}, \\'longitude\\': {{\\'title\\': \\'Longitude\\', "
 "\\'type\\': \\'string\\'}}}}\\n\\nUse a json blob to specify a tool by "
 'providing an action key (tool name) and an action_input key (tool '
 'input).\\n\\nValid "action" values: "Final Answer"

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
llm_agent.invoke({"input": "Can you check the weather in Zurich for me?"})



> Entering new AgentExecutor chain...
Let me check on that for you.

Thought: To look up the weather in Zurich, I first need to get the latitude and longitude coordinates for Zurich. I can use the get_lat_long tool for that.

Action:
```json
{
  "action": "get_lat_long",
  "action_input": "Zurich"
}
```


Observation: {'latitude': '47.3744489', 'longitude': '8.5410422'}
Thought:Now that I have the latitude and longitude for Zurich, I can use the get_weather tool to get the current weather data for that location.

Action:
```json  
{
  "action": "get_weather",
  "action_input": {
    "latitude": "47.3744489",
    "longitude": "8.5410422"
  }
}
```


Observation: {'latitude': 47.38, 'longitude': 8.539999, 'generationtime_ms': 0.048995018005371094, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 416.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 

{'input': 'Can you check the weather in Zurich for me?',
 'output': 'Here is the current weather in Zurich based on the latitude and longitude I looked up:\n\nThe current temperature is 14.7°C. The wind speed is 5.1 km/h from the south (176°). The weather code indicates broken clouds. This data is as of 7:45 PM local Zurich time.  \n\nLet me know if you need any other weather information for Zurich!',
 'intermediate_steps': [(AgentAction(tool='get_lat_long', tool_input='Zurich', log='Let me check on that for you.\n\nThought: To look up the weather in Zurich, I first need to get the latitude and longitude coordinates for Zurich. I can use the get_lat_long tool for that.\n\nAction:\n```json\n{\n  "action": "get_lat_long",\n  "action_input": "Zurich"\n}\n```\n\n'),
   {'latitude': '47.3744489', 'longitude': '8.5410422'}),
  (AgentAction(tool='get_weather', tool_input={'latitude': '47.3744489', 'longitude': '8.5410422'}, log='Now that I have the latitude and longitude for Zurich, I can use

# 2. Create retriever tool for RAG (retrieval augmented generation)

## 2.1 Create Vector DB in memory with Amazon SageMaker FAQ as content
The file can be [found here](./rag_data/Amazon_SageMaker_FAQs.pdf)

In [16]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain.memory import ConversationBufferMemory

loader = PyPDFLoader("./rag_data/Amazon_SageMaker_FAQs.pdf")
bedrock_client = boto3_bedrock
texts = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(loader.load())
embed_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)
#- create the vector store
db = FAISS.from_documents(texts, embed_model)

retriever = db.as_retriever(search_kwargs={"k": 4})
tool_search = create_retriever_tool(
    retriever=retriever,
    name="search_sagemaker_policy",
    description="Searches and returns excerpts for any question about SageMaker",
)
pprint(tool_search.func)
tool_search.args_schema.schema()

functools.partial(<function _get_relevant_documents at 0x7f8815fd1750>, retriever=VectorStoreRetriever(tags=['FAISS', 'BedrockEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f880845b400>, search_kwargs={'k': 4}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')


{'title': 'RetrieverInput',
 'description': 'Input to the retriever.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'query to look up in retriever',
   'type': 'string'}},
 'required': ['query']}

# Task 2: Based on the implementation from above, please add the retriever functionality as tool and initialize the agent again 

In [17]:
# write your code here, no imports are needed. Take the retriever tool from above and integrate it into the LLM, to be able to interact with the SageMaker FAQ document

# add the retriever tool to our toolset
tools_list.append(tool_search)

# Construct the Tools agent and also the executor
llm_agent = initialize_agent(
    tools_list, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
)

## Let's test the retriever tool

In [18]:
# After finishing, test your new tool by running this cell
llm_agent.invoke({"input": "What is a Amazon SageMaker pipeline?"})



> Entering new AgentExecutor chain...
Let me provide some information on Amazon SageMaker pipelines:

Thought: To answer the question about what a SageMaker pipeline is, I will first get an overview summary by using the search_sagemaker_policy tool.

Action:
```json
{
  "action": "search_sagemaker_policy",
  "action_input": "what is a sagemaker pipeline"
}
```


Observation: goals. You can track data, code, trained models, and more with SageMaker Pipelines."How does the pricing for Amazon SageMaker Pipelines work?,There is no additional charge for Amazon SageMaker Pipelines. You pay only for the underlying compute or any separate AWS services you use within SageMaker Pipelines.Can I use Kubeflow with Amazon SageMaker?,"Yes. Amazon SageMaker Components for Kubeflow Pipelines are open-source plugins that allow you to use Kubeflow Pipelines to define your ML workflows and use SageMaker for the data labeling, training, and inference steps. Kubeflow Pipelines is an add-on to Kubeflow that

{'input': 'What is a Amazon SageMaker pipeline?',
 'output': 'Amazon SageMaker Pipelines is a service that helps you build, automate, manage and track end-to-end ML workflows. It provides a visual interface to build pipelines that connect data processing, training, evaluation and deployment steps. SageMaker Pipelines keeps track of all data, parameters and results for full traceability of workflows. It generates reports to observe pipeline progress and comes integrated with other SageMaker components. SageMaker Pipelines helps manage thousands of production models and automates execution of each step in a workflow.',
 'intermediate_steps': [(AgentAction(tool='search_sagemaker_policy', tool_input='what is a sagemaker pipeline', log='Let me provide some information on Amazon SageMaker pipelines:\n\nThought: To answer the question about what a SageMaker pipeline is, I will first get an overview summary by using the search_sagemaker_policy tool.\n\nAction:\n```json\n{\n  "action": "search_

In [20]:
# Let's check whether the weather tool is still working
llm_agent.invoke({"input": "How is the weather in New York"})



> Entering new AgentExecutor chain...
Let me check on that for you.

Thought: To look up the weather in New York, I first need to get the latitude and longitude coordinates for New York using the get_lat_long tool. 

Action:
```
{
  "action": "get_lat_long",
  "action_input": "New York" 
}
```


Observation: {'latitude': '40.7127281', 'longitude': '-74.0060152'}
Thought:Now that I have the latitude and longitude for New York, I can use the get_weather tool to get the current weather data for those coordinates.

Action:
```
{
  "action": "get_weather",
  "action_input": "40.7127281", 
  "action_input": "-74.0060152"
}
```



ValidationError: 1 validation error for get_weatherSchema
longitude
  field required (type=value_error.missing)

# 3. Build a dog image tool

# Task 3: Build a tool for the LLM that fetches images from different dog breeds.

In [ ]:
# Please use the Dog API (https://dog.ceo/dog-api/documentation), and more specifically the random breed endpoint (https://dog.ceo/dog-api/breeds-list)
# Based on a breed input, the LLM should have a tool available to call the API
# at the end of the notebook, you can find code to display the image, the task is done, if you can display a random image based on the LLM output

In [ ]:
@tool ("get_dog_image")
def get_dog_image(breed: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    response = requests.get(f"https://dog.ceo/api/breed/{breed.lower()}/images/random")

    if response:
        return response.json()
    else:
        return None

In [ ]:
# add the retriever tool to our toolset
tools_list.append(get_dog_image)

# Construct the Tools agent and also the executor
llm_agent = initialize_agent(
    tools_list, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
)

In [ ]:
chain_output = llm_agent.invoke({"input": "give me a labrador image"})
chain_output

## Display the dog image

In [ ]:
if chain_output:
    output_str = chain_output["output"]
    url_pattern = r'(https?://[^\s]+)'
    url = re.findall(url_pattern, output_str)[0]
    print(url)
else:
    raise TypeError("Output cannot be of NoneType")

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the image from the response content
    img = Image.open(BytesIO(response.content))
    
    # Display the image in the notebook
    display(img)
else:
    print("Failed to retrieve the image")
